In [63]:
import pandas as pd
import xgboost as xgb
import numpy as np

In [64]:
spammer_order = "../feature.json"
raw_file = pd.read_json(open(spammer_order), orient='records')
raw_file.head()

,active_day_num,active_day_ratio,comment_num,content_length,followers_num,following_num,forward_weibo_num,forwarded_num,hashtag,id,is_spammer,mention,post_num,reputation,text_similarity,time_interval_mean,time_interval_var,upvote_num,url
0,15,0.365854,16,60.480000,179,358,38,0,15,1214054254,no,7,1005,0.500000,0.034592,30972.857143,8.239935e+08,9,1
1,11,0.423077,2,48.510204,312,180,49,0,10,2158838150,no,3,1819,1.733333,0.026582,33023.000000,7.491242e+08,0,0
2,5,0.625000,44,59.979592,704,715,40,0,18,1465038162,no,3,3542,0.984615,0.027306,36453.937500,6.229419e+08,9,2
3,9,0.310345,0,102.660000,5,387,43,0,27,5203407567,no,19,340,0.012920,0.031989,11511.346939,7.102748e+08,1,4
4,15,0.077320,0,54.918367,93,455,0,0,7,3944654540,yes,3,371,0.204396,0.055030,26378.770833,7.995144e+08,0,38


In [65]:
raw_file.describe()

,active_day_num,active_day_ratio,comment_num,content_length,followers_num,following_num,forward_weibo_num,forwarded_num,hashtag,id,mention,post_num,reputation,text_similarity,time_interval_mean,time_interval_var,upvote_num,url
count,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1.566000e+03,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1.566000e+03,1566.000000,1566.000000
mean,11.479566,0.387625,21.282886,65.434807,558.009579,683.523627,24.169221,27.308429,10.814815,2.966758e+09,6.033206,1460.061941,2.009261,0.062515,29051.851426,7.228954e+08,24.281609,7.306513
std,6.599973,0.385907,64.078166,22.656497,1701.318891,626.339048,17.258607,369.338919,9.773090,1.185777e+09,7.110370,3472.917546,28.911290,0.091577,10882.853761,2.586875e+08,103.198753,10.694027
min,1.000000,0.000000,0.000000,5.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.006765e+08,0.000000,2.000000,0.009146,0.020051,6.204082,0.000000e+00,0.000000,0.000000
25%,6.000000,0.085868,1.000000,49.580833,133.250000,220.000000,7.000000,0.000000,3.000000,2.021067e+09,1.000000,294.750000,0.317523,0.030094,22165.185566,6.211736e+08,1.000000,1.000000
50%,11.000000,0.268762,4.000000,62.160000,246.000000,412.000000,24.000000,1.000000,8.000000,2.727800e+09,4.000000,669.000000,0.579435,0.037440,30547.811437,7.618443e+08,6.000000,3.000000
75%,16.000000,0.571429,21.000000,78.075306,496.500000,970.000000,41.000000,4.000000,16.000000,3.540640e+09,8.000000,1520.750000,1.051232,0.055187,37380.515306,8.798472e+08,20.000000,8.000000
max,34.000000,2.000000,1249.000000,181.250000,33924.000000,3229.000000,50.000000,10704.000000,49.000000,5.397886e+09,50.000000,102353.000000,1129.000000,1.000000,65396.666667,1.731597e+09,2795.000000,50.000000


In [66]:
data = raw_file.drop(['is_spammer'], axis=1)
target = raw_file[['is_spammer']]


In [67]:
data

,active_day_num,active_day_ratio,comment_num,content_length,followers_num,following_num,forward_weibo_num,forwarded_num,hashtag,id,mention,post_num,reputation,text_similarity,time_interval_mean,time_interval_var,upvote_num,url
0,15,0.365854,16,60.480000,179,358,38,0,15,1214054254,7,1005,0.500000,0.034592,30972.857143,8.239935e+08,9,1
1,11,0.423077,2,48.510204,312,180,49,0,10,2158838150,3,1819,1.733333,0.026582,33023.000000,7.491242e+08,0,0
2,5,0.625000,44,59.979592,704,715,40,0,18,1465038162,3,3542,0.984615,0.027306,36453.937500,6.229419e+08,9,2
3,9,0.310345,0,102.660000,5,387,43,0,27,5203407567,19,340,0.012920,0.031989,11511.346939,7.102748e+08,1,4
4,15,0.077320,0,54.918367,93,455,0,0,7,3944654540,3,371,0.204396,0.055030,26378.770833,7.995144e+08,0,38
5,4,0.571429,1,50.800000,253,1850,0,2,0,5055635073,1,467,0.136757,0.030800,37345.510204,1.169251e+08,17,0
6,7,0.875000,0,73.931818,59,120,40,0,6,5037076803,2,620,0.491667,0.051178,18005.395349,1.072603e+09,2,2
7,8,0.008502,0,59.680000,116,384,0,0,4,2703197560,42,401,0.302083,0.415591,14511.469388,6.036607e+08,0,4
8,13,0.158537,0,61.840000,257,516,44,0,4,2172215857,1,631,0.498062,0.029096,20287.081633,8.519779e+08,2,3
9,14,0.583333,11,69.666667,6323,1810,10,193,12,1223815707,4,1479,3.493370,0.074021,33078.620000,1.000798e+09,35,18


In [68]:
target

,is_spammer
0,no
1,no
2,no
3,no
4,yes
5,yes
6,no
7,yes
8,no
9,yes


In [69]:
from sklearn import preprocessing

def norm(data, column_name):
    x = data[column_name].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    data_norm = pd.DataFrame(x_scaled)
    data[column_name] = data_norm
    return data

In [70]:
norm(data, 'time_interval_mean')
norm(data, 'time_interval_var')

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(

,active_day_num,active_day_ratio,comment_num,content_length,followers_num,following_num,forward_weibo_num,forwarded_num,hashtag,id,mention,post_num,reputation,text_similarity,time_interval_mean,time_interval_var,upvote_num,url
0,15,0.365854,16,60.480000,179,358,38,0,15,1214054254,7,1005,0.500000,0.034592,0.473565,0.475858,9,1
1,11,0.423077,2,48.510204,312,180,49,0,10,2158838150,3,1819,1.733333,0.026582,0.504918,0.432620,0,0
2,5,0.625000,44,59.979592,704,715,40,0,18,1465038162,3,3542,0.984615,0.027306,0.557386,0.359750,9,2
3,9,0.310345,0,102.660000,5,387,43,0,27,5203407567,19,340,0.012920,0.031989,0.175945,0.410185,1,4
4,15,0.077320,0,54.918367,93,455,0,0,7,3944654540,3,371,0.204396,0.055030,0.403309,0.461721,0,38
5,4,0.571429,1,50.800000,253,1850,0,2,0,5055635073,1,467,0.136757,0.030800,0.571021,0.067524,17,0
6,7,0.875000,0,73.931818,59,120,40,0,6,5037076803,2,620,0.491667,0.051178,0.275257,0.619430,2,2
7,8,0.008502,0,59.680000,116,384,0,0,4,2703197560,42,401,0.302083,0.415591,0.221825,0.348615,0,4
8,13,0.158537,0,61.840000,257,516,44,0,4,2172215857,1,631,0.498062,0.029096,0.310150,0.492019,2,3
9,14,0.583333,11,69.666667,6323,1810,10,193,12,1223815707,4,1479,3.493370,0.074021,0.505768,0.577963,35,18


In [71]:
target

,is_spammer
0,no
1,no
2,no
3,no
4,yes
5,yes
6,no
7,yes
8,no
9,yes


In [72]:
from sklearn.model_selection import GridSearchCV
target['is_spammer']=target['is_spammer'].map(dict(yes=1, no=0))

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


# 2. Machine Learning Model

## 2.1 Classical Models

In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

In [74]:
svc = SVC(kernel='rbf', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=111)
abc = AdaBoostClassifier(n_estimators=62, random_state=111)
bc = BaggingClassifier(n_estimators=9, random_state=111)
etc = ExtraTreesClassifier(n_estimators=9, random_state=111)

In [75]:
clfs = {'SVC' : svc,'KNeighborsClassifier' : knc, 'MultinomialNaiveBayes': mnb, 'DecisionTreeClassifier': dtc, 
        'LogisticRegression': lrc, 'RandomForestClassifier': rfc, 'AdaBoost': abc, 'BaggingClassifier': bc, 
        'ExtraTreesClassifier': etc}

In [76]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=5)

# from sklearn.model_selection import ShuffleSplit
# cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
from sklearn.utils import column_or_1d
target = column_or_1d(target, warn=True)
pred_scores = []
for k,v in clfs.items():
    results = cross_val_score(v, data, target, cv=cv)
    print(k, results, results.mean())
    pred_scores.append((k, results))

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


DecisionTreeClassifier [ 0.69745223  0.7028754   0.68370607  0.68690096  0.69329073] 0.692845078448
BaggingClassifier [ 0.77070064  0.74440895  0.73482428  0.76038339  0.70607029] 0.74327750758
SVC [ 0.47133758  0.52715655  0.50479233  0.52715655  0.50159744] 0.506408091003
ExtraTreesClassifier [ 0.74203822  0.77955272  0.71565495  0.76996805  0.67092652] 0.735628090596
LogisticRegression [ 0.77070064  0.7827476   0.76677316  0.76996805  0.70607029] 0.759251948475
AdaBoost [ 0.76751592  0.76996805  0.75079872  0.75079872  0.72204473] 0.752225229442
MultinomialNaiveBayes [ 0.64649682  0.58466454  0.57507987  0.65175719  0.61022364] 0.613644410981
KNeighborsClassifier [ 0.61783439  0.66453674  0.61980831  0.66773163  0.5942492 ] 0.6328320547
RandomForestClassifier [ 0.76433121  0.77955272  0.76677316  0.79233227  0.73801917] 0.768201705297


In [77]:
df = pd.DataFrame.from_items(pred_scores,orient='index', columns=['Score'])
df

,Score
DecisionTreeClassifier,0.697452
BaggingClassifier,0.770701
SVC,0.471338
ExtraTreesClassifier,0.742038
LogisticRegression,0.770701
AdaBoost,0.767516
MultinomialNaiveBayes,0.646497
KNeighborsClassifier,0.617834
RandomForestClassifier,0.764331


In [78]:
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)
        folds = list(KFold(len(y), n_folds=self.n_folds, shuffle=True, random_state=2016))
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((T.shape[0], len(folds)))
            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                # y_holdout = y[test_idx]
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]
            S_test[:, i] = S_test_i.mean(1)
        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict(S_test)[:]
        return y_pred

In [79]:
e = Ensemble(5, clfs['RandomForestClassifier'], clfs)


In [80]:
e.fit_predict(data, target, target)

TypeError: __init__() got an unexpected keyword argument 'n_folds'

## 2.2 XGBoost

In [81]:
# split to train and test sets
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(
    data, target, test_size=0.33, random_state=0)

train_X = train_X.as_matrix()
test_X = test_X.as_matrix()
train_y = train_y.as_matrix()

AttributeError: 'numpy.ndarray' object has no attribute 'as_matrix'

In [82]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)
gbm.fit(train_X, train_y)
predictions = gbm.predict(test_X)

In [83]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 75.05%


In [84]:
# or use 10 fold cross validation
from sklearn.model_selection import cross_val_score
model = xgb.XGBClassifier()

from sklearn.model_selection import KFold
cv = KFold(n_splits=5)

# from sklearn.model_selection import ShuffleSplit
# cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)

results = cross_val_score(model, data, target, cv=cv)
print(results)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
# initial mean accuracy 48.54%, deviation 31.44%

[ 0.79936306  0.79872204  0.74440895  0.7827476   0.75079872]
Accuracy: 77.52% (2.34%)


# 3. Parameter Tuning 

## 3.1 xgboost model parameters

In [85]:
def cross_val(alg):
#     kfold = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
    kfold = KFold(n_splits=5, random_state=0)
    scoring = ['accuracy', 'recall']
    results = cross_val_score(alg, data, target, cv=kfold, scoring='accuracy')
    print(results)
    print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    # initial mean accuracy 48.54%, deviation 31.44%

### 3.1.1. Find the number of estimators for a high learning rate

In [86]:
from xgboost.sklearn import XGBClassifier
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
cross_val(xgb1)

[ 0.77070064  0.76677316  0.75399361  0.78594249  0.73162939]
Accuracy: 76.18% (1.82%)


In [87]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}


from sklearn.model_selection import ShuffleSplit
shuffle = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)

gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=True, cv=shuffle)
gsearch1.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'min_child_weight': range(1, 6, 2), 'max_depth': range(3, 10, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [88]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.83745, std: 0.01697, params: {'min_child_weight': 1, 'max_depth': 3},
  mean: 0.84026, std: 0.01782, params: {'min_child_weight': 3, 'max_depth': 3},
  mean: 0.84198, std: 0.01342, params: {'min_child_weight': 5, 'max_depth': 3},
  mean: 0.83887, std: 0.01546, params: {'min_child_weight': 1, 'max_depth': 5},
  mean: 0.83833, std: 0.01879, params: {'min_child_weight': 3, 'max_depth': 5},
  mean: 0.83607, std: 0.01783, params: {'min_child_weight': 5, 'max_depth': 5},
  mean: 0.84039, std: 0.01767, params: {'min_child_weight': 1, 'max_depth': 7},
  mean: 0.83479, std: 0.01826, params: {'min_child_weight': 3, 'max_depth': 7},
  mean: 0.84026, std: 0.01851, params: {'min_child_weight': 5, 'max_depth': 7},
  mean: 0.83890, std: 0.01316, params: {'min_child_weight': 1, 'max_depth': 9},
  mean: 0.83700, std: 0.01480, params: {'min_child_weight': 3, 'max_depth': 9},
  mean: 0.83659, std: 0.01535, params: {'min_child_weight': 5, 'max_depth': 9}],
 {'max_depth': 3, 'min_child_weight': 5

In [89]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test2 = {
    'max_depth':[2,3,4],
    'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=3,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch2.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=5, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'min_child_weight': [4, 5, 6], 'max_depth': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [90]:
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.84450, std: 0.01677, params: {'min_child_weight': 4, 'max_depth': 2},
  mean: 0.84645, std: 0.01748, params: {'min_child_weight': 5, 'max_depth': 2},
  mean: 0.84754, std: 0.01652, params: {'min_child_weight': 6, 'max_depth': 2},
  mean: 0.84152, std: 0.01535, params: {'min_child_weight': 4, 'max_depth': 3},
  mean: 0.84198, std: 0.01342, params: {'min_child_weight': 5, 'max_depth': 3},
  mean: 0.84119, std: 0.01602, params: {'min_child_weight': 6, 'max_depth': 3},
  mean: 0.83988, std: 0.01795, params: {'min_child_weight': 4, 'max_depth': 4},
  mean: 0.83806, std: 0.01923, params: {'min_child_weight': 5, 'max_depth': 4},
  mean: 0.84057, std: 0.02179, params: {'min_child_weight': 6, 'max_depth': 4}],
 {'max_depth': 2, 'min_child_weight': 6},
 0.84753909486570966)

In [91]:
param_test2b = {
    'min_child_weight':[6,8,10,12, 14, 16]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch2b.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'min_child_weight': [6, 8, 10, 12, 14, 16]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [92]:
gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.84754, std: 0.01652, params: {'min_child_weight': 6},
  mean: 0.84738, std: 0.01479, params: {'min_child_weight': 8},
  mean: 0.84841, std: 0.01780, params: {'min_child_weight': 10},
  mean: 0.84555, std: 0.01673, params: {'min_child_weight': 12},
  mean: 0.84544, std: 0.01525, params: {'min_child_weight': 14},
  mean: 0.84485, std: 0.01492, params: {'min_child_weight': 16}],
 {'min_child_weight': 10},
 0.84840926870667988)

In [55]:
param_test2c = {
    'min_child_weight':[16, 20, 24, 28, 32, 36, 40]
}
gsearch2c = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2c, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch2c.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'min_child_weight': [16, 20, 24, 28, 32, 36, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [56]:
gsearch2c.grid_scores_, gsearch2c.best_params_, gsearch2c.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.83864, std: 0.02118, params: {'min_child_weight': 16},
  mean: 0.83590, std: 0.02170, params: {'min_child_weight': 20},
  mean: 0.83235, std: 0.02159, params: {'min_child_weight': 24},
  mean: 0.83188, std: 0.02152, params: {'min_child_weight': 28},
  mean: 0.83208, std: 0.02165, params: {'min_child_weight': 32},
  mean: 0.83165, std: 0.02467, params: {'min_child_weight': 36},
  mean: 0.83302, std: 0.02440, params: {'min_child_weight': 40}],
 {'min_child_weight': 16},
 0.83863933967770909)

In [93]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=10, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch3.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=10, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [94]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.84841, std: 0.01780, params: {'gamma': 0.0},
  mean: 0.84841, std: 0.01780, params: {'gamma': 0.1},
  mean: 0.84841, std: 0.01780, params: {'gamma': 0.2},
  mean: 0.84841, std: 0.01780, params: {'gamma': 0.3},
  mean: 0.84841, std: 0.01780, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 0.84840926870667988)

In [95]:
xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=7,
        min_child_weight=36,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
# modelfit(xgb2, data)
cross_val(xgb2)

[ 0.7866242   0.77316294  0.74121406  0.81469649  0.70926518]
Accuracy: 76.50% (3.65%)


### Tune subsample and colsample_bytree

In [98]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=10, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch4.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=10, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'colsample_bytree': [0.6, 0.7, 0.8, 0.9], 'subsample': [0.6, 0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [99]:
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.84333, std: 0.01724, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.84458, std: 0.01657, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.84945, std: 0.01735, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.84826, std: 0.01476, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.84320, std: 0.01646, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.84589, std: 0.01632, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.84675, std: 0.01616, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.84728, std: 0.01399, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.84329, std: 0.01793, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.84554, std: 0.01441, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.84841, std: 0.01780, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.84755, std: 0.01400, params: {'colsample_bytree': 0.8, 'subsample'

### regularization

In [35]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
                                        min_child_weight=10, gamma=0.4, subsample=0.7, colsample_bytree=0.6,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=shuffle)
gsearch6.fit(data,target)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=0, test_size=0.3, train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.6,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=6, missing=None, n_estimators=140, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.7),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'reg_alpha': [1e-05, 0.01, 0.1, 1, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [36]:
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

/home/lzc/Common/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.82955, std: 0.00549, params: {'reg_alpha': 1e-05},
  mean: 0.82901, std: 0.00474, params: {'reg_alpha': 0.01},
  mean: 0.82887, std: 0.00646, params: {'reg_alpha': 0.1},
  mean: 0.82877, std: 0.00832, params: {'reg_alpha': 1},
  mean: 0.50000, std: 0.00000, params: {'reg_alpha': 100}],
 {'reg_alpha': 1e-05},
 0.82955393083374318)

In [37]:
from sklearn import cross_validation
xgb = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=2,
                    min_child_weight=6, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
                    objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)
kf = cross_validation.KFold(len(target), n_folds=5)
for train_index, test_index in kf:

   X_train, X_test = data[train_index], data[test_index]
   y_train, y_test = target[train_index], target[test_index]

   xgb.fit(X_train, y_train)
   print(confusion_matrix(y_test, xgb.predict(X_test)))

KeyError: '[ 314  315  316 ..., 1563 1564 1565] not in index'

In [38]:
target['is_spammer'].value_counts()
data.describe()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Others

In [39]:
target.value_counts()

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [40]:
param_test = {
    'gamma':np.linspace(0.05, 1, num=20),
    'kernel':['rbf', 'poly', 'sigmoid']
}
cv = KFold(n_splits=5)
gsearch = GridSearchCV(estimator = SVC(kernel='sigmoid', gamma=1), 
                       param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=cv)
gsearch.fit(data, target)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

KeyboardInterrupt: 

In [ ]:
# split to train and test sets
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(
    data, target, test_size=0.33, random_state=0)

train_X = train_X.as_matrix()
test_X = test_X.as_matrix()
# train_y = train_y.as_matrix()

pred_scores = []
krnl = {'rbf' : 'rbf','polynominal' : 'poly', 'sigmoid': 'sigmoid'}
for k,v in krnl.items():
    for i in np.linspace(0.05, 1, num=20):
        svc = SVC(kernel=v, gamma=i)
        svc.fit(train_X, train_y)
        pred = svc.predict(test_X)
        pred_scores.append((k, [i, accuracy_score(test_y,pred)]))
df = pd.DataFrame.from_items(pred_scores,orient='index', columns=['Gamma','Score'])
df['Score'].plot(kind='line', figsize=(11,6), ylim=(0.8,1.0))

In [ ]:
df[df['Score'] == df['Score'].max()]